## Water box

This workflow demonstrates how to use MoSDeF to prepare, run, and briefly analyze an MD simulation of water

## Install requirements

In [ ]:
!conda install --file requirements.txt --yes

If `gmx` (the `GROMACS` executable) is not found, install it via `conda`.

In [ ]:
!if ! [ -x "$(command -v gmx)" ]; then conda install -c bioconda gromacs; fi

In [ ]:
import mbuild
import foyer

## Preparing the system coordinates

After importing our libraries, can load in a water molecule from disk. Then, the packing function `fill_box` calls `PACKMOL` under the hood to place copies of this molecule in a cubic box.

In [ ]:
water = mb.load('files/spce.mol2')
water.name = 'water'

In [ ]:
water_box = mb.fill_box(
    compound=water,     
    n_compounds=3000,
    density=1000,
)

## Applying the force field

We use `Foyer` to load the SPC/E force field, in XML format, and apply it to our system.

In [ ]:
spce = Forcefield('files/spce.xml')
system = spce.apply(water_box.to_parmed(residues=['water']))

Finally, we save out two of the files we need to run a `GROMACS` simulation. The GRO file stores atomic coordinates and the TOP file stores force field information.

In [ ]:
system.save('system.gro', combine='all')
system.save('system.top', combine='all')

## Running the simulation

We need to call `grompp` to prepare the simulation and then run it with `mdrun`

In [ ]:
!gmx grompp -f files/npt.mdp -c system.gro -p system.top -o npt.tpr

In [ ]:
!gmx mdrun -v -deffnm npt

# Analyzing the simulation

Here we use `MDTraj` to load trajectory data from disk and make a quick plot of the density over time compared to reference data.

In [ ]:
import mdtraj as md
import numpy as np

trj = md.load('npt.xtc', top='system.gro')

In [ ]:
rho = md.density(trj)

rho_ref = np.loadtxt('ref_data/rho.txt')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()

ax.plot(trj.time, rho, 'b-', label='Data')
ax.plot(rho_ref[:, 0], rho_ref[:, 1], 'k-', label='Reference')
ax.legend()
ax.set_xlabel('Simulation time, ps')
ax.set_ylabel('System density, kg/m^3')